<a href="https://colab.research.google.com/github/Gcarmnonapy7/Masterpiece_prototype_scheduling_movie_theaters/blob/main/movies_cluesterization_%26crowd_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
import os
path_tmdb = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")
path_movie_box = kagglehub.dataset_download("aditya126/movies-box-office-dataset-2000-2024")

Using Colab cache for faster access to the 'tmdb-movie-metadata' dataset.


100%|██████████| 216k/216k [00:00<00:00, 21.3MB/s]

Extracting files...


In [2]:
print(os.listdir(path_tmdb))
print(os.listdir(path_movie_box))

['tmdb_5000_movies.csv', 'tmdb_5000_credits.csv']
['enhanced_box_office_data(2000-2024)u.csv']


In [3]:
#Import datasets
import pandas as pd

tmdb = pd.read_csv(os.path.join(path_tmdb,'tmdb_5000_movies.csv'))
box = pd.read_csv(os.path.join(path_movie_box,'enhanced_box_office_data(2000-2024)u.csv'))

Predicted Crowd Week w=∑similarityi​∑(similarityi​×crowdi,w​)​

In [4]:
print(tmdb.shape)
print(box.shape)

(4803, 20)
(5000, 13)


In [ ]:
columns_to_rename = {'Release Group': 'title','$Worldwide':'worldwide_gross','Rating' : 'rating','Production_Countries' : 'producer_country','Original_Language':'language'}

In [ ]:
box.rename(columns={columns_to_rename}) # title and total crowd around the world

In [ ]:
# === functions to help ===

from collections import Counter

def extract_countries(x):

  if pd.isna(x) or x == ' ':
    return ''

  countries = json.loads(x)

  return ([c['iso_3166_1'] for c in countries])

all_countries = [c for c in box['producer_country']]
top_10_countries = [c for c , _ in Counter(all_countries).most_common(10)]


In [ ]:
# === EDA ===

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')

In [9]:
# Cleaning TMDB ==> FOR FEATURE SIMILARITY

import json
import numpy as np

def clean_genres(col):
  if pd.isna(col) or col == '':
    return ''

  genres = json.loads(col)

  return ' '.join([g['name'].replace(' ','_').lower() for g in genres])

tmdb['genres'] = tmdb['genres'].apply(clean_genres)
tmdb['release_year'] = pd.to_datetime(tmdb['release_date']).dt.year

In [10]:
tmdb = tmdb[['title','genres','runtime','vote_average','vote_count','popularity','release_year']].dropna()

In [11]:
tmdb.head()

,title,genres,runtime,vote_average,vote_count,popularity,release_year
0,Avatar,action adventure fantasy science_fiction,162.0,7.2,11800,150.437577,2009.0
1,Pirates of the Caribbean: At World's End,adventure fantasy action,169.0,6.9,4500,139.082615,2007.0
2,Spectre,action adventure crime,148.0,6.3,4466,107.376788,2015.0
3,The Dark Knight Rises,action crime drama thriller,165.0,7.6,9106,112.312950,2012.0
4,John Carter,action adventure science_fiction,132.0,6.1,2124,43.926995,2012.0


In [23]:
box = box[['title','worldwide_gross']]

KeyError: "None of [Index(['title', 'worldwide_gross'], dtype='object')] are in the [columns]"